In [1]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, CodeInterpreterTool, CodeInterpreterToolAuto

load_dotenv()

foundary_project_endpoint = os.getenv("FOUNDARY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")


In [5]:
project_client = AIProjectClient(
    endpoint = foundary_project_endpoint,
    credential = DefaultAzureCredential(),
)

In [6]:
openai_client = project_client.get_openai_client()

In [8]:
# Upload the csv file for the code interpreter to use

file = openai_client.files.create(purpose="assistants", file=open("./electronics_products.csv", "rb"))
print(f"File uplaoded (id: {file.id})")

File uplaoded (id: assistant-3wQu7CjK6ikBCr2MQJU4nh)


In [10]:
agent_name = "code-interpreter-agent"

agent = project_client.agents.create_version(
    agent_name=agent_name,
    definition = PromptAgentDefinition(
        model=model_deployment_name,
        instructions="You are a helpful AI Assistant with code interpreter capabilities.",
        tools=[
            CodeInterpreterTool(
                container = CodeInterpreterToolAuto(
                    file_ids = [file.id]
                )
            )
        ]
    )
)

# printing the agent id 
print(f'Agent created (id : {agent.id}, name: {agent.name}, version: {agent.version})')

Agent created (id : code-interpreter-agent:1, name: code-interpreter-agent, version: 1)


In [11]:
conversation = openai_client.conversations.create()
print(f"Created conversations with id: {conversation.id}")

Created conversations with id: conv_512875790f6180d100w9DNH5mMCRsrbm7JcmxdYjZelBIFu5oq


In [16]:
## Calling the agent

response = openai_client.responses.create(
    conversation = conversation.id,
    input = "Could you please create a column chart with products on the x-axis and their respective prices on the y-axis?",
    extra_body = {
        "agent": {
            "name": agent_name,
            "type": "agent_reference"
        }
    }

)

print(f"Response completed with id: {response.id}")

Response completed with id: resp_512875790f6180d100695500126010819093502aa635068638


In [17]:
## Extracting the file which is saved as output

file_id = ""
filename = ""
container_id = ""

# Get the last message which should contain file citations
last_message = response.output[-1]  # ResponseOutputMessage
if last_message.type == "message":
        # Get the last content item (contains the file annotations)
        text_content = last_message.content[-1]  # ResponseOutputText
        if text_content.type == "output_text":
            # Get the last annotation (most recent file)
            if text_content.annotations:
                file_citation = text_content.annotations[-1]  # AnnotationContainerFileCitation
                if file_citation.type == "container_file_citation":
                    file_id = file_citation.file_id
                    filename = file_citation.filename
                    container_id = file_citation.container_id
                    print(f"Found generated file: {filename} (ID: {file_id})")

# Download the generated file if available
if file_id and filename:
        file_content = openai_client.containers.files.content.retrieve(file_id=file_id, container_id=container_id)
        with open(filename, "wb") as f:
            f.write(file_content.read())
            print(f"File {filename} downloaded successfully.")
        print(f"File ready for download: {filename}")
else:
        print("No file generated in response")

Found generated file: cfile_69550018ee3c8190b61ae565036e9c85.png (ID: cfile_69550018ee3c8190b61ae565036e9c85)
File cfile_69550018ee3c8190b61ae565036e9c85.png downloaded successfully.
File ready for download: cfile_69550018ee3c8190b61ae565036e9c85.png
